In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import warnings
warnings.filterwarnings('ignore')


In [2]:
# TensorFlow and Keras imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers, callbacks
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [18]:
data_dir = "data"
img_size = (224, 224)
img_channels = 3
batch_size = 32

In [19]:
train = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=True,
    validation_split=0.2,
    subset='training',
    seed=50)
test = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    seed=50)


Found 154 files belonging to 3 classes.
Using 124 files for training.


Found 154 files belonging to 3 classes.
Using 30 files for validation.


In [41]:
train.class_names

['man1', 'man2', 'man3']

In [21]:
normalization_layer = layers.Rescaling(1./255)

In [7]:
augmentation_layer = keras.Sequential([
    layers.RandomRotation(factor=30/360,fill_mode='constant',fill_value=0.0),
    layers.RandomZoom(height_factor=0.2,width_factor=0.2,fill_mode='constant',fill_value=0.0),
    layers.RandomBrightness(factor=[0.8,1.1]),
    layers.RandomContrast(factor=0.2),
    layers.RandomTranslation(height_factor=0.1,width_factor=0.1,fill_mode='constant',fill_value=0.0)
], name="augmentation_layer")


In [37]:
def preprocess_train(data,label):
    data = normalization_layer(data)
    data = augmentation_layer(data, training=True)
    return data,label

def preprocess_test(data,label):
    data = normalization_layer(data)
    return data,label

In [38]:
train_x = train.map(preprocess_train)
test_x = test.map(preprocess_test)

In [47]:
train_x

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [8]:
base_model = MobileNetV2(
            input_shape=img_size + (img_channels,),
            include_top=False,           # Remove ImageNet classification head
            weights='imagenet',          # Pre-trained on ImageNet
            pooling='avg'                # Global average pooling output
        )

In [9]:
base_model.trainable = False

In [40]:
trainable_count = sum([layer.trainable for layer in base_model.layers])
print(f"   Frozen layers: {len(base_model.layers) - trainable_count}")
print(f"   Trainable layers: {trainable_count}")

   Frozen layers: 155
   Trainable layers: 0


In [10]:
from tensorflow.keras import regularizers

In [56]:
input = layers.Input(shape=img_size + (img_channels,),name="input_layer")

x = base_model(input, training=False)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2, name='dropout_final')(x)
output = layers.Dense(len(train.class_names), activation='softmax', name='output_layer')(x)

In [57]:
model = keras.Model(inputs=input, outputs=output)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_final (Dropout)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [58]:
optimizer = keras.optimizers.Adam(
            learning_rate=0.001,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07
        )
loss = keras.losses.SparseCategoricalCrossentropy()
metrics = [
            'accuracy'
        ]

In [59]:
model.compile(
            optimizer=optimizer,
            loss=loss,
            metrics=metrics
        )

In [60]:
hist = model.fit(train_x, epochs=10, validation_data=test_x)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 40s 5s/step - accuracy: 0.3871 - loss: 1.2628 - val_accuracy: 0.5667 - val_loss: 0.9424
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.2903 - loss: 1.2453 - val_accuracy: 0.5000 - val_loss: 0.9234
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 20s 5s/step - accuracy: 0.3629 - loss: 1.2767 - val_accuracy: 0.6333 - val_loss: 0.8829
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 4s/step - accuracy: 0.2742 - loss: 1.2625 - val_accuracy: 0.6000 - val_loss: 0.8846
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.3871 - loss: 1.1794 - val_accuracy: 0.6000 - val_loss: 0.9135
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step - accuracy: 0.3548 - loss: 1.1446 - val_accuracy: 0.6667 - val_loss: 0.8934
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.3871 - loss: 1.1855 - val_accuracy: 0.5000 - val_loss: 0.9874
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.3710 - loss: 1.1477 - val_accuracy: 0.5000 - val_loss: 1.0215
Epoch 9/

In [67]:
hist.history['accuracy']

[0.3870967626571655,
 0.29032257199287415,
 0.3629032373428345,
 0.27419355511665344,
 0.3870967626571655,
 0.35483869910240173,
 0.3870967626571655,
 0.3709677457809448,
 0.3709677457809448,
 0.32258063554763794]